# 量子振幅推定

量子振幅増幅は、ある量子状態の振幅を増幅するアルゴリズムでした。  

量子振幅推定は、その名の通りある量子状態の振幅を推定するアルゴリズムです。このような操作は、量子振幅増幅と同じオラクルを用いて行うことができます。


Amplitude amplification の説明に用いた以下の図を再度用います。

<img width="30%" src="https://upload.wikimedia.org/wikipedia/commons/1/16/Grovers_algorithm_geometry.png">

参考: https://en.wikipedia.org/wiki/Grover%27s_algorithm

$\lvert s\rangle$ について振幅推定を行うことを考えましょう。  
$\lvert s\rangle$ を次のように書き直します。

$$
\lvert s\rangle = \sqrt{1-a} \lvert \omega_{\perp}\rangle - \sqrt{a}\lvert\omega\rangle\ \ \ \ (\lvert \omega_{\perp}\rangle = \lvert s'\rangle)
$$

求めたいのは $\lvert \omega\rangle$ の振幅 $\sqrt{a}$ です。

$U_{\omega}$, $U_s$ はそれぞれ次のように書けます。

$$
U_{\omega} = I - 2\lvert \omega \rangle \langle \omega \rvert \\
U_{s} = 2\lvert s \rangle \langle s \rvert - I
$$

ここで天下り的ですが、振幅増幅に用いたオラクル $U_s U_{\omega}$ は以下のような固有ベクトル $\lvert\psi_{\pm}\rangle$、固有値 $\lambda_{\pm}$ を持ちます。

$$
\lvert\psi_{\pm}\rangle = \frac{1}{\sqrt{2}} (\lvert\omega_{\perp}\rangle \mp i\lvert\omega\rangle) \\
\lambda_{\pm} = e^{i2\theta_a}\ \ \ \ (\sin{\theta_a} = \sqrt{a})
$$

$\theta_a$ が得られれば $\lvert \omega\rangle$ の振幅 $\sqrt{a}$ を求めることができます。  
ここでは量子位相推定を用いて $\theta_a$ を求めます。  
量子位相推定は、ユニタリ行列(ここでは$U_s U_{\omega}$)の固有値 $e^{i\theta}$ における $\theta$ (位相)を求めるアルゴリズムです (※詳細については量子位相推定のチュートリアルを参照)。  

量子位相推定では、求めたい固有値に対応する固有状態(もしくはその近似状態)を用意する必要があります。  
性質の良いことに、振幅推定したい状態 $\lvert s\rangle$ は固有状態同士を重ね合わせた状態として書けることが、計算により確認できます。

$$
\lvert s\rangle = \frac{1}{\sqrt{2}} \bigl(e^{i\theta_a} \lvert \psi_{+}\rangle + e^{-i\theta_a} \lvert \psi_{-}\rangle\bigr)
$$

量子位相推定を行うと、どちらかの固有状態に対応する固有値の位相が、確率的に得られます。  
この場合は $2\theta_a$ または $-2\theta_a$ が得られますが、最終的に求めたい値は $\sin{\theta_a} = \sqrt{a}$ です。  
$\sin (\theta_a) = \sin (-\theta_a)$ より、どちらの場合でも振幅を推定することができます。

よって、振幅推定したい状態 $\lvert s\rangle$ と、それに対する振幅増幅オラクルが用意できれば振幅推定が可能になります。  
以下がアルゴリズムの手順です。

1. 振幅を推定したい状態 $\lvert s\rangle$ を用意する
2. オラクル $U_s U_{\omega}$ を用意する
3. $U_s U_{\omega}$ の固有値を量子位相推定により求め、推定したい振幅を計算する

これをblueqatで実装してみましょう。


振幅増幅のチュートリアルと同様の、2量子ビットの例を考えます。  
2量子ビットの状態は、00,01,10,11の4通りです。その中から特定の状態の振幅を推定したいと思います。  

In [8]:
from blueqat import Circuit
import numpy as np

以下では量子位相推定を行うため、振幅増幅で用いたオラクルの制御ゲート版を用意します。

In [9]:
Us = Circuit(2).h[:].x[:].cz[0,1].x[:].h[:]
Uw00 = Circuit(2).s[:].cz[0,1].s[:]
Uw01 = Circuit(2).s[1].cz[0,1].s[1] 
Uw10 = Circuit(2).s[0].cz[0,1].s[0]
Uw11 = Circuit(2).cz[0,1]

def C_Us(qc, c, t1, t2):
    qc.ch[c, t1].ch[c, t2].cx[c, t1].cx[c, t2]
    qc.cx[t1, t2].tdg[t2].cx[c, t2].t[t2].cx[t1, t2].tdg[t2].cx[c, t2].t[t1].t[t2].cx[c, t1].t[c].tdg[t1].cx[c, t1]
    qc.cx[c, t1].cx[c, t2].ch[c, t1].ch[c, t2]
    
def C_Uw00(qc, c, t1, t2):
    qc.cphase(np.pi/2)[c, t1].cphase(np.pi/2)[c, t2]
    qc.cx[t1, t2].tdg[t2].cx[c, t2].t[t2].cx[t1, t2].tdg[t2].cx[c, t2].t[t1].t[t2].cx[c, t1].t[c].tdg[t1].cx[c, t1]
    qc.cphase(np.pi/2)[c, t1].cphase(np.pi/2)[c, t2]
    
def C_Uw01(qc, c, t1, t2):
    qc.cphase(np.pi/2)[c, t2]
    qc.cx[t1, t2].tdg[t2].cx[c, t2].t[t2].cx[t1, t2].tdg[t2].cx[c, t2].t[t1].t[t2].cx[c, t1].t[c].tdg[t1].cx[c, t1]
    qc.cphase(np.pi/2)[c, t2]
    
def C_Uw10(qc, c, t1, t2):
    qc.cphase(np.pi/2)[c, t1]
    qc.cx[t1, t2].tdg[t2].cx[c, t2].t[t2].cx[t1, t2].tdg[t2].cx[c, t2].t[t1].t[t2].cx[c, t1].t[c].tdg[t1].cx[c, t1]
    qc.cphase(np.pi/2)[c, t1]
    
def C_Uw11(qc, c, t1, t2):
    qc.cx[t1, t2].tdg[t2].cx[c, t2].t[t2].cx[t1, t2].tdg[t2].cx[c, t2].t[t1].t[t2].cx[c, t1].t[c].tdg[t1].cx[c, t1]

次に、振幅増幅オラクルの固有値を量子位相推定によって求めます。

In [10]:
import math

# 逆量子フーリエ変換関数を用意
def qft_rotate_single_inv(circuit, i, n):
    if n == 0:
        return circuit
    for qubit in range(0, i):
        circuit.cphase(-np.pi/2**(i - qubit))[n - 1 - qubit, n - 1 - i]
    circuit.h[n - 1 - i]

def qft_dagger(circuit, n):
    for i in range(math.floor(n/2)):
        circuit.swap[i, n - (i + 1)]
    for i in range(n):
        qft_rotate_single_inv(circuit, i, n)

In [15]:
n_encode = 4 # 求めたい固有値の位相角をエンコードする量子ビット数
n_eigstate = 2 # 固有状態の量子ビット数
n = n_encode + n_eigstate

qc = Circuit(n)
qc.h[n_encode].h[n_encode + 1] # 固有状態を用意

for qubit in range(n_encode):
    qc.h[qubit]

repetitions = 1
for count in reversed(range(n_encode)):
    for i in range(repetitions):
        C_Uw00(qc, count, n_encode, n_encode + 1)
        C_Us(qc, count, n_encode, n_encode + 1)
        #qc.cp(theta, count, n_encode)
    repetitions *= 2

qft_dagger(qc, n_encode)

for n in range(n_encode):
    qc.m[n]

In [22]:
res = qc.run(shots = 1024)
for key in res.keys():
    print(key[:n_encode], ':', res[key])

1101 : 342
0010 : 90
0011 : 371
1100 : 26
1110 : 87
0100 : 21
0110 : 8
1111 : 16
0000 : 10
1001 : 3
1010 : 8
0111 : 6
0001 : 13
0101 : 14
1011 : 9


今回振幅推定した状態は4状態の等しい重ね合わせです。よって各状態($\lvert 00\rangle, \lvert 01\rangle, \lvert 10\rangle, \lvert 11\rangle$)はそれぞれ振幅 $1/2$ を持ちます。  

よって、初期状態 $\lvert s\rangle = \sqrt{1-a} \lvert \omega_{\perp}\rangle - \sqrt{a}\lvert\omega\rangle$ において $\sqrt{a} = 1/2$ 、$\theta_a = \pi / 6$ です。

振幅増幅に用いたオラクルは固有値　$\exp(\pm i 2\theta) = \exp(\pm i\pi / 3)$ を持つため、$\frac{\pm\pi/3}{2\pi} = \pm\frac{1}{6}$  が量子位相推定で期待する出力です。

上の例では量子位相推定の出力を4量子ビットでエンコードしているため、出力は4bit精度で得られます。  
測定結果では '0011' が $\frac{3}{2^{4}} = 0.1875$ を表しており、$\frac{1}{6} = 0.166...$ に最も近い値になります。  
これにより $\theta$ が近似的に求まります。  

以上より量子振幅推定を行うことができました。